# Token Swap Using Marlowe Runtime\'s REST API

The token-swap contract example is a simple Marlowe contract that lets parties trade ada for a token.

[A video works through this Jupyter notebook.](https://youtu.be/sSrVCRNoytU)

In this demonsration we use Marlowe Runtime\'s REST API, served via `marlowe-web-server`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use role tokens and we have Marlowe Runtime mint them.

In [Marlowe Playground](https://play.marlowe-finance.io/), the contract looks like this in Blockly format.

![Marlowe contract for swapping tokens](images/swap-playground.png)

In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Role "Ada provider")
            (Role "Ada provider")
            (Token "" "")
            (MulValue
                (Constant 1000000)
                (ConstantParam "Amount of Ada")
            )
        )
        (When
            [Case
                (Deposit
                    (Role "Dollar provider")
                    (Role "Dollar provider")
                    (Token "9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe" "Djed_testMicroUSD")
                    (ConstantParam "Amount of dollars")
                )
                (Pay
                    (Role "Ada provider")
                    (Party (Role "Dollar provider"))
                    (Token "" "")
                    (MulValue
                        (Constant 1000000)
                        (ConstantParam "Amount of Ada")
                    )
                    (Pay
                        (Role "Dollar provider")
                        (Party (Role "Ada provider"))
                        (Token "9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe" "Djed_testMicroUSD")
                        (ConstantParam "Amount of dollars")
                        Close 
                    )
                )]
            (TimeParam "Timeout for dollar deposit")
            Close 
        )]
    (TimeParam "Timeout for Ada deposit")
    Close
```

## Preliminaries

See [Lesson 0. Preliminaries](00-preliminaries.md) for information on setting up one's environment for using this tutorial.

The lesson assumes that the following environment variables have been set.
- `CARDANO_NODE_SOCKET_PATH`: location of Cardano node's socket.
- `CARDANO_TESTNET_MAGIC`: testnet magic number.
- `MARLOWE_RT_WEBSERVER_HOST`: IP address of the Marlowe Runtime web server.
- `MARLOWE_RT_WEBSERVER_PORT`: Port number for the Marlowe Runtime web server.

It also assumes that the parties have addresses, signing keys, and funds.
- Ada Provider
    - [keys/lender.address](keys/lender.address): Cardano address for the ada provider
    - [keys/lender.skey](keys/lender.skey): location of signing key file for the ada provider
- Dollar Provider
    - [keys/borrower.address](keys/borrower.address): Cardano address for the dollar provider
    - [keys/borrower.skey](keys/borrower.skey): location of signing key file for the dollar provider

### Access to Cardano node and Marlowe Runtime

If we're using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension, then we already have access to Cardano Node and Marlowe Runtime. The followind commands will set the required environment variables to use a local docker deployment on the default ports. It will also set some supplementary environment variables.

In [1]:
if [[ -z "$MARLOWE_RT_WEBSERVER_PORT" ]]
then

  if command -v podman > /dev/null
  then
    DOCKER_CLI=podman
  else
    DOCKER_CLI=docker
  fi
  
  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$($DOCKER_CLI volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

  # Only required for Marlowe Runtime REST API.
  export MARLOWE_RT_WEBSERVER_HOST="127.0.0.1"
  export MARLOWE_RT_WEBSERVER_PORT=3780

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

MARLOWE_RT_WEBSERVER_URL="http://$MARLOWE_RT_WEBSERVER_HOST:$MARLOWE_RT_WEBSERVER_PORT"

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"
echo "MARLOWE_RT_WEBSERVER_HOST = $MARLOWE_RT_WEBSERVER_HOST"
echo "MARLOWE_RT_WEBSERVER_PORT = $MARLOWE_RT_WEBSERVER_PORT"
echo "MARLOWE_RT_WEBSERVER_URL = $MARLOWE_RT_WEBSERVER_URL"

CARDANO_NODE_SOCKET_PATH = /home/bbush/.local/share/containers/storage/volumes/marlowe-starter-kit_shared/_data/node.socket
CARDANO_TESTNET_MAGIC = 1
MARLOWE_RT_WEBSERVER_HOST = 127.0.0.1
MARLOWE_RT_WEBSERVER_PORT = 3780
MARLOWE_RT_WEBSERVER_URL = http://127.0.0.1:3780


### Ada Provider address and funds

Check that an address and key has been created for the ada provider. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [2]:
ADA_PROVIDER_SKEY=keys/lender.skey
ADA_PROVIDER_ADDR=$(cat keys/lender.address)
echo "ADA_PROVIDER_ADDR = $ADA_PROVIDER_ADDR"

ADA_PROVIDER_ADDR = addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


Check that the ada provider has at least three hundred ada.

In [3]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
81fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51     1        1000000000 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [4]:
echo "$EXPLORER_URL/address/$ADA_PROVIDER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


### Dollar Provider address and funds

Check that an address and key has been created for the dollar provider. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [5]:
USD_PROVIDER_SKEY=keys/borrower.skey
USD_PROVIDER_ADDR=$(cat keys/borrower.address)
echo "USD_PROVIDER_ADDR = $USD_PROVIDER_ADDR"

USD_PROVIDER_ADDR = addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


Check that the dollar provider has at least one hundred USD tokens.

In [6]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
81fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51     2        1000000000 lovelace + TxOutDatumNone
81fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51     3        2000000 lovelace + 100000000 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe.446a65645f746573744d6963726f555344 + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [7]:
echo "$EXPLORER_URL/address/$USD_PROVIDER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


## Design the contract

The swap contract can be downloaded from the [Marlowe Playground](https://play.marlowe-finance.io/) as a JSON file, or it can be generated using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) using the `marlowe-cli template` command.

Set the ada amount to 294 and the dollar amount to 100.000000.

In [8]:
ADA=1000000  # 1 ada = 1,000,000 lovelace
LOVELACE_AMOUNT=$((294 * ADA))
MICROUSD_AMOUNT=$((100 * 1000000))
echo "LOVELACE_AMOUNT = $LOVELACE_AMOUNT lovelace"
echo "MICROUSD_AMOUNT = $MICROUSD_AMOUNT µUSD"

LOVELACE_AMOUNT = 294000000 lovelace
MICROUSD_AMOUNT = 100000000 µUSD


On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [9]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

MIN_LOVELACE = 2000000 lovelace


Later in the example we will need some constants for converting times.

In [10]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

Now design the contract.

1. Visit https://play.marlowe-finance.io/ in a web browser.
2. Select "Open an Example".
3. Select "Marlowe" or "Blockly" under "Swap".
4. Edit the contract so that the dollar token has the policy ID and name for the dollar token. In this example we use test djed, which has policy ID `9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe` and token name `Djed_testMicroUSD`.
5. Select "Send to Simulator".
6. Set the "Timeout for ada deposit" to one hour into the future.
7. Set the "Timeout for dollar deposit" to two hours into the future
8. Set "Amount of Ada" to 294.
9. Set "Amount of dollars" to 100,000,000, since the units of measure are millionths.
10. Select "Download as JSON", set the file name to "swap-contract.json", and store the file in this folder, namely [marlowe-starter-kit/05-swap-rest/](.).

![Setting parameters for the swap contract in Marlowe Playground](images/swap-simulation.png)

## Examine the contract

View the contract file as YAML.

In [11]:
json2yaml swap-contract.json

timeout: 1679669100000
timeout_continuation: close
when:
- case:
    deposits:
      multiply: 1000000
      times: 294
    into_account:
      role_token: Ada provider
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Ada provider
  then:
    timeout: 1679673000000
    timeout_continuation: close
    when:
    - case:
        deposits: 100000000
        into_account:
          role_token: Dollar provider
        of_token:
          currency_symbol: 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe
          token_name: Djed_testMicroUSD
        party:
          role_token: Dollar provider
      then:
        from_account:
          role_token: Ada provider
        pay:
          multiply: 1000000
          times: 294
        then:
          from_account:
            role_token: Dollar provider
          pay: 100000000
          then: close
          to:
            party:
              role_token: Ada provider
          token:
           

### \[Optional\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe-finance.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe-finance.io/) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

See [Lesson 1](01-runtime-cli.ipynb) for an example of performing step 6.

## Transaction 1: Ada provider Creates Swap Contract with Initial ADA

A `HTTP` `POST` request to Marlowe Runtime\'s `/contracts` endpoint will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

First we create the JSON body of the request to build the creation transaction.

In [12]:
yaml2json << EOI > request-1.json
version: v1
contract: $(cat swap-contract.json)
roles:
  Ada provider: $ADA_PROVIDER_ADDR
  Dollar provider: $USD_PROVIDER_ADDR
minUTxODeposit: $MIN_LOVELACE
metadata: {}
tags: {}
EOI
cat request-1.json

{"contract":{"timeout":1679669100000,"timeout_continuation":"close","when":[{"case":{"deposits":{"multiply":1000000,"times":294},"into_account":{"role_token":"Ada provider"},"of_token":{"currency_symbol":"","token_name":""},"party":{"role_token":"Ada provider"}},"then":{"timeout":1679673000000,"timeout_continuation":"close","when":[{"case":{"deposits":100000000,"into_account":{"role_token":"Dollar provider"},"of_token":{"currency_symbol":"9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe","token_name":"Djed_testMicroUSD"},"party":{"role_token":"Dollar provider"}},"then":{"from_account":{"role_token":"Ada provider"},"pay":{"multiply":1000000,"times":294},"then":{"from_account":{"role_token":"Dollar provider"},"pay":100000000,"then":"close","to":{"party":{"role_token":"Ada provider"}},"token":{"currency_symbol":"9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe","token_name":"Djed_testMicroUSD"}},"to":{"party":{"role_token":"Dollar provider"}},"token":{"currency_symbol":"",

Next we post the request and view the response.

In [13]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $ADA_PROVIDER_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

links:
  contract: contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231
resource:
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  txBody:
    cborHex: 86a8008182582081fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51010d8182582081fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51010184a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a3b54cd37a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e84800282005820bf610358ef3035e032f179bffee0de88c48478d4aef7b7b2bcb0d3a7351092bda200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a00102da4a1581cbb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4a14c4164612070726f766964657201a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a00106026a1581cbb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4a14f446f6c6c61722070726f76696465720110a200581d601b120d7221aa5e7db58fd8e0753e156cf8c3

The identifier for the contract is embedded in the response.

In [14]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1


The CBOR serialization (in text-envelope format) is also embedded in the response.

In [15]:
jq '.resource.txBody' response-1.json > tx-1.unsigned

There are many ways to sign and submit Cardano transactions:
- `cardano-cli` at the command line
- `cardano-wallet` at the command line or as a REST service
- `cardano-hw-cli` for a hardware wallet at the command line
- a Babbage-compatible CIP-30 wallet in a web browser
- `marlowe-cli` at the command line

For convenience, here we use `marlowe-cli transaction submit`. One may have to wait a minute or so for the transactions to be confirmed on the blockchain.

In [16]:
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$ADA_PROVIDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

TX_1 = c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef


One can view the transaction on a Cardano explorer and see that the contract has been created and the parties have received their role tokens. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [17]:
echo "$EXPLORER_URL/transaction/$TX_1?tab=utxo"

https://preprod.cardanoscan.io/transaction/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef?tab=utxo


In particular, we see that the Marlowe contract holds the 2 ada that was set as `MINIMUM_LOVELACE`.

In [18]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$CONTRACT_ID"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "bf610358ef3035e032f179bffee0de88c48478d4aef7b7b2bcb0d3a7351092bd"


One can see that the ada and dollar providers have received their role tokens. Note that `4164612070726f7669646572 = "Ada provider`, `4275796572 = Buyer`, and `446f6c6c61722070726f7669646572 = "Dollar provider"` in hexadecimal notation. Also note that `446a65645f746573744d696372 = Djed_testMicroUSD`.

In [19]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef     0        995413303 lovelace + TxOutDatumNone
c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef     2        1060260 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.4164612070726f7669646572 + TxOutDatumNone


In [20]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
81fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51     2        1000000000 lovelace + TxOutDatumNone
81fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51     3        2000000 lovelace + 100000000 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe.446a65645f746573744d6963726f555344 + TxOutDatumNone
c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef     3        1073190 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.446f6c6c61722070726f7669646572 + TxOutDatumNone


## View the details of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}` can fetch a contract from the blockchain and return information about it.

In [21]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/$(jq -r '.links.contract' response-1.json)"
echo "CONTRACT_URL = $CONTRACT_URL"

CONTRACT_URL = http://127.0.0.1:3780/contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231


In [22]:
curl -sS "$CONTRACT_URL" | json2yaml

links:
  transactions: contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231/transactions
resource:
  block:
    blockHeaderHash: 65a52894d0f20909681734f5b5f461e0e24fdc79c9115b8f2b127856ed754378
    blockNo: 757491
    slotNo: 23982361
  continuations: null
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  currentContract:
    timeout: 1679669100000
    timeout_continuation: close
    when:
    - case:
        deposits:
          multiply: 1000000
          times: 294
        into_account:
          role_token: Ada provider
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: Ada provider
      then:
        timeout: 1679673000000
        timeout_continuation: close
        when:
        - case:
            deposits: 100000000
            into_account:
              role_token: Dollar provider
            of_token:
              currency_symbol: 9772ff715b691c0444f333ba1db

## Transaction 2: The Ada Provider Deposits 294 Ada into the Contract

The ada provider deposits their 294 ada into the contract using Marlowe Runtime\'s `HTTP` `POST` `/contract/{contractId}/transactions` endpoint. The buyer is providing the funding for and receiving the change from this transaction, so we provide their address.

The deposit is represented as JSON input to the contract. The `marlowe-cli input deposit` tool conveniently formats the correct JSON for a deposit.

In [23]:
marlowe-cli input deposit \
  --deposit-party 'Ada provider' \
  --deposit-account 'Ada provider' \
  --deposit-amount "$LOVELACE_AMOUNT" \
  --out-file input-2.json
json2yaml input-2.json

input_from_party:
  role_token: Ada provider
into_account:
  role_token: Ada provider
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 294000000


In [24]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
tags: {}
EOI
cat request-2.json

{"inputs":[{"input_from_party":{"role_token":"Ada provider"},"into_account":{"role_token":"Ada provider"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":294000000}],"metadata":{},"tags":{},"version":"v1"}


Next we post the request and store the response.

In [25]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $ADA_PROVIDER_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
json2yaml response-2.json

links:
  transaction: contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231/transactions/84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58
resource:
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  transactionId: 84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58
  txBody:
    cborHex: 86aa0083825820c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef00825820c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef01825820c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef020d81825820c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a29c39bd0a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a11a49a000282005820c945bedf448b262fe3e5c36cf3033a4a4914456277f0de38432c4f28bd352ed3a200581d601b120

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [26]:
jq '.resource.txBody' response-2.json > tx-2.unsigned

In [27]:
TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$ADA_PROVIDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = 84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [28]:
echo "$EXPLORER_URL/transaction/$TX_2?tab=utxo"

https://preprod.cardanoscan.io/transaction/84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58?tab=utxo


One can see that the ada provider has approximately 294 ada less than originally.

In [29]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58     0        700685264 lovelace + TxOutDatumNone
84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58     2        1060260 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.4164612070726f7669646572 + TxOutDatumNone


The Marlowe contract still has the 2 ada from its creation and an additional 294 ada.

In [30]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58     1        296000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "c945bedf448b262fe3e5c36cf3033a4a4914456277f0de38432c4f28bd352ed3"


## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [31]:
curl -sS "$CONTRACT_URL/transactions/$TX_2" | json2yaml

links: {}
resource:
  block:
    blockHeaderHash: a8e43ba8381f9c177cb788476bb3143ae122b9251e04d5f6fa74260a9b36e122
    blockNo: 757501
    slotNo: 23982522
  consumingTx: null
  continuations: null
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  inputUtxo: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  inputs:
  - input_from_party:
      role_token: Ada provider
    into_account:
      role_token: Ada provider
    of_token:
      currency_symbol: ''
      token_name: ''
    that_deposits: 294000000
  invalidBefore: 2023-03-24T13:48:07Z
  invalidHereafter: 2023-03-24T14:45:00Z
  metadata: {}
  outputContract:
    timeout: 1679673000000
    timeout_continuation: close
    when:
    - case:
        deposits: 100000000
        into_account:
          role_token: Dollar provider
        of_token:
          currency_symbol: 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe
          token_name: Djed_testMicroUSD
        party:
 

## Transaction 3: The Dollar Provider deposits 100 Djed into the Contract

The dollar provider deposits their 100 djed into the contract using Marlowe Runtime\'s `HTTP` `POST` `/contract/{contractId}/transactions` endpoint. The buyer is providing the funding for and receiving the change from this transaction, so we provide their address.

The deposit is represented as JSON input to the contract. The `marlowe-cli input deposit` tool conveniently formats the correct JSON for a deposit.

In [32]:
marlowe-cli input deposit \
  --deposit-party 'Dollar provider' \
  --deposit-account 'Dollar provider' \
  --deposit-token 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe.Djed_testMicroUSD \
  --deposit-amount "$MICROUSD_AMOUNT" \
  --out-file input-3.json
json2yaml input-3.json

input_from_party:
  role_token: Dollar provider
into_account:
  role_token: Dollar provider
of_token:
  currency_symbol: 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe
  token_name: Djed_testMicroUSD
that_deposits: 100000000


In [33]:
yaml2json << EOI > request-3.json
version: v1
inputs: [$(cat input-3.json)]
metadata: {}
tags: {}
EOI
cat request-3.json

{"inputs":[{"input_from_party":{"role_token":"Dollar provider"},"into_account":{"role_token":"Dollar provider"},"of_token":{"currency_symbol":"9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe","token_name":"Djed_testMicroUSD"},"that_deposits":100000000}],"metadata":{},"tags":{},"version":"v1"}


Next we post the request and store the response.

In [34]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $USD_PROVIDER_ADDR" \
  -d @request-3.json \
  -o response-3.json \
  -sS
json2yaml response-3.json

links:
  transaction: contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231/transactions/05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e
resource:
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  transactionId: 05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e
  txBody:
    cborHex: 86aa008482582081fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb510282582081fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb510382582084e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e5801825820c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef030d8182582081fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb510212818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010185a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a3b98370aa200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a00106026a1581cbb7053ce0

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [35]:
jq '.resource.txBody' response-3.json > tx-3.unsigned

In [36]:
TX_3=$(
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$USD_PROVIDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

TX_3 = 05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [37]:
echo "$EXPLORER_URL/transaction/$TX_3?tab=utxo"

https://preprod.cardanoscan.io/transaction/05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e?tab=utxo


One can see that the dollar provider has exactly 100 djed less than originally.

In [38]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e     0        999831306 lovelace + TxOutDatumNone
05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e     1        1073190 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.446f6c6c61722070726f7669646572 + TxOutDatumNone


The Marlowe contract has closed, but the roll-payout address holds 100 djed for the benefit of the ada provider.

In [39]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_3#2"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e     2        1258520 lovelace + 100000000 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe.446a65645f746573744d6963726f555344 + TxOutDatumHash ScriptDataInBabbageEra "758d98b7ec8818a2cd32baa416d6a337bcb837a2393815744e3f4935bebedd3e"


The roll-payout address also holds 294 ada for the benefit of the dollar provider.

In [40]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_3#3"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e     3        294000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "366c6e0ba1da1c0fe3488c49fba1f8dcdd6f4dcc723bbd08bc62caf12633facc"


## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [41]:
curl -sS "$CONTRACT_URL/transactions/$TX_3" | json2yaml

links:
  previous: contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231/transactions/84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58
resource:
  block:
    blockHeaderHash: c331402d98a3e77d8d78123a915118e39e4acef61613e57cae46cc047d796fb7
    blockNo: 757509
    slotNo: 23982639
  consumingTx: null
  continuations: null
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  inputUtxo: 84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58#1
  inputs:
  - input_from_party:
      role_token: Dollar provider
    into_account:
      role_token: Dollar provider
    of_token:
      currency_symbol: 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe
      token_name: Djed_testMicroUSD
    that_deposits: 100000000
  invalidBefore: 2023-03-24T13:50:04Z
  invalidHereafter: 2023-03-24T15:50:00Z
  metadata: {}
  outputContract: null
  outputState: null
  outputUtxo: null
  status: confirmed
  tags: {}
  transacti

## Transaction 4: The Ada Provider Withdraws the Dollars

The 100 djed is held at Marlowe's role-payout address for the benefit of the ada provider. The ada provider can withdraw these funds at any time. The contract ID and role name are included in the request body for a withdrawal.

In [42]:
yaml2json << EOI > request-4.json
contractId: "$CONTRACT_ID"
role: "Ada provider"
EOI
cat request-4.json

{"contractId":"c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1","role":"Ada provider"}


Next we post the request and store the response.

In [43]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $ADA_PROVIDER_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -sS
json2yaml response-4.json

links:
  withdrawal: withdrawals/cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37
resource:
  txBody:
    cborHex: 86a8008382582005b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e0282582084e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e580082582084e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58020d8182582084e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e580012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd020183a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a29c08003a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a00102da4a1581cbb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4a14c4164612070726f766964657201a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a0010c52aa1581c9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fea151446a65645f746573744d6963726f5553441a05f5e10010a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [44]:
jq '.resource.txBody' response-4.json > tx-4.unsigned

In [45]:
TX_4=$(
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer "$ADA_PROVIDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"

TX_4 = cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37


On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [46]:
echo "$EXPLORER_URL/transaction/$TX_4?tab=utxo"

https://preprod.cardanoscan.io/transaction/cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37?tab=utxo


The ada provider now has the 100 djed.

In [47]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e     4        2000000 lovelace + TxOutDatumNone
cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37     0        700481539 lovelace + TxOutDatumNone
cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37     1        1060260 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.4164612070726f7669646572 + TxOutDatumNone
cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37     2        1099050 lovelace + 100000000 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe.446a65645f746573744d6963726f555344 + TxOutDatumNone


## View the withdrawal

Marlowe Runtime\'s `HTTP` `GET` endpoint `/withdrawals/{transactionId}` can fetch a withdrawal from the blockchain and return information about it.

In [48]:
curl -sS "$MARLOWE_RT_WEBSERVER_URL/withdrawals/$TX_4" | json2yaml

block:
  blockHeaderHash: b41c4d2ed3de99126ddfbd379c6e6f4387cbf5bbc8af744a04b66754280bd3ed
  blockNo: 757514
  slotNo: 23982857
payouts:
- contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  payout: 05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e#2
  role: Ada provider
  roleTokenMintingPolicyId: bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4
status: confirmed
withdrawalId: cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37


## Transaction 5: The Dollar Provider Withdraws the Ada

The 294 ada is held at Marlowe's role-payout address for the benefit of the dollar provider. The dollar provider can withdraw these funds at any time. The contract ID and role name are included in the request body for a withdrawal.

In [49]:
yaml2json << EOI > request-5.json
contractId: "$CONTRACT_ID"
role: "Dollar provider"
EOI
cat request-5.json

{"contractId":"c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1","role":"Dollar provider"}


Next we post the request and store the response.

In [50]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $USD_PROVIDER_ADDR" \
  -d @request-5.json \
  -o response-5.json \
  -sS
json2yaml response-5.json

links:
  withdrawal: withdrawals/315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867
resource:
  txBody:
    cborHex: 86a8008382582005b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e0082582005b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e0182582005b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e030d8182582005b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd020182a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a4d193a50a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a00106026a1581cbb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4a14f446f6c6c61722070726f76696465720110a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a3b909bb3111a00079b57021a0005123a0b5820424a94276d7c3699508f073d8809325917434fbf05bd0b48f82aba66a55997c49fff81d8799f581cbb7053ce07dda2292f779c92ae789b4c4a99376f5f6

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [51]:
jq '.resource.txBody' response-5.json > tx-5.unsigned

In [52]:
TX_5=$(
marlowe-cli transaction submit \
  --tx-body-file tx-5.unsigned \
  --required-signer "$USD_PROVIDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_5 = $TX_5"

TX_5 = 315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867


On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [53]:
echo "$EXPLORER_URL/transaction/$TX_5?tab=utxo"

https://preprod.cardanoscan.io/transaction/315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867?tab=utxo


The dollar provider now has about an additional 294 ada.

In [54]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867     0        1293498960 lovelace + TxOutDatumNone
315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867     1        1073190 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.446f6c6c61722070726f7669646572 + TxOutDatumNone


## View the withdrawal

Marlowe Runtime\'s `HTTP` `GET` endpoint `/withdrawals/{transactionId}` can fetch a withdrawal from the blockchain and return information about it.

In [55]:
curl -sS "$MARLOWE_RT_WEBSERVER_URL/withdrawals/$TX_5" | json2yaml

block:
  blockHeaderHash: f4a1cfc8471034fcc5f58008909c429e8f9d0bdd8efdb93ab29d2e62b7a0bfa0
  blockNo: 757519
  slotNo: 23982974
payouts:
- contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  payout: 05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e#3
  role: Dollar provider
  roleTokenMintingPolicyId: bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4
status: confirmed
withdrawalId: 315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867
